<div style="text-align: right" align="right"><i>Peter Norvig<br> 2019, revised 2024<br>Based on <a href="https://nbviewer.org/gist/yoavg/d76121dfde2618422139">Yoav Goldberg's 2015 notebook</a></i></div> 

# The Effectiveness of Generative Language Models

This notebook is an expansion of [**Yoav Goldberg's 2015 notebook**](https://nbviewer.org/gist/yoavg/d76121dfde2618422139) on character-level *n*-gram language models, which in turn was a response to  [**Andrej Karpathy's 2015 blog post**](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) on recurrent neural network (RNN) language models. 

The term [**generative AI**](https://en.wikipedia.org/wiki/Generative_artificial_intelligencehttps://en.wikipedia.org/wiki/Generative_artificial_intelligence) is all the rage these days; it refers to computer programs that can *generate* something new (such as an image or a piece of text). 

In 2015 Karpathy's point was that recurrent neural networks were unreasonably effective at generating good text, even though they are at heart rather simple. Goldberg's point was that, yes, they are effective, but actually most of the magic is not in the RNNs, it is  in the training data itself, and an even simpler model (with no neural nets) does just as well at generating English text. Goldberg and Karpathy agree that the RNN captures some aspects of C++ code that the simpler model does not.


## Definitions

- A **generative language model** is a model that, when given an initial text, can predict what tokens come next; it can generate a continuation of a partial text. (And when the initial text is empty, it can generate the whole text.) In terms of probabilities, the model represents *P*(*t* | *h*), the probability distribution that the next token will be *t*, given a history of previous tokens *h*. The probability distribution is estimated by looking at a training corpus of text.

- A **token** is a unit of text. It can be a single character (as covered by Karpathy and Goldberg) or more generally it can be a word or a part of a word (as allowed in my implementation).

- A generative model stands in contrast to a **discriminative model**, such as an email spam filter, which can discriminate between spam and non-spam, but can't be used to generate a new sample of spam.


- An ***n*-gram model** is a generative model that estimates the probability of *n*-token sequences. For example, a 5-gram character model would be able to say that given the previous 4 characters `'chai'`, the next character might be `'r'` or `'n'` (to form `'chair'` or `'chain'`). A 5-gram model is also called a model of **order** 4, because it maps from the 4 previous tokens to the next token.

- A **recurrent neural network (RNN) model** is more powerful than an *n*-gram model, because it contains memory units that allow it to retain information from more than *n* tokens. See Karpathy for [details](http://karpathy.github.io/2015/05/21/rnn-effectiveness/).

- Current **large language models** such as Chat-GPT, Claude, Llama, and Gemini use an even more powerful model called a [transformer](https://en.wikipedia.org/wiki/Transformer_%28deep_learning_architecture%29).  Karpathy has [an introduction](https://www.youtube.com/watch?v=zjkBMFhNj_g&t=159s).

## Training Data

A language model learns probabilities by observing a corpus of text that we call the **training data**. 

Both Karpathy and Goldberg use the works of Shakespeare (about 800,000 words) as their initial training data:

In [1]:
! [ -f shakespeare_input.txt ] || curl -O https://norvig.com/ngrams/shakespeare_input.txt
! wc shakespeare_input.txt 
# Print the number of lines, words, and characters

  167204  832301 4573338 shakespeare_input.txt


In [2]:
! head -8 shakespeare_input.txt 
# First 8 lines

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?


## Python Code for n-Gram Model

I do some imports and then define two data types:
- A `Token` is an individual unit of text, a string of one or more characters.
- A `LanguageModel` is a subclass of `defaultdict` that maps a history of length `order` tokens to a `Counter` of the number of times each token appears immediately following the history in the training data.

In [3]:
import random
from typing import *
from collections import defaultdict, Counter

Token = str # Datatype to represent a token

class LanguageModel(defaultdict): 
    """A mapping of {'history': Counter(next_token)}."""
    def __init__(self, order):
        self.order = order
        super().__init__(Counter)

I define two main functions that do essentially all the work:

- `train_LM` takes a sequence of tokens (the training data) and an integer `order`, and builds a language model, formed by counting the times each token *t* occurs and storing that under the entry for the history *h* of `order` tokens that precede *t*. 
- `generate_tokens` generates a random sequence of tokens. At each step it looks at the history of previously generated tokens and chooses a new token at random from the language model's counter for that history.

In [4]:
def train_LM(tokens, order: int) -> LanguageModel:
    """Create and train a language model of given order on the given tokens."""
    LM = LanguageModel(order)
    history = []
    for token in tokens:
        LM[cat(history)][token] += 1
        history = (history + [token])[-order:] 
    return LM

def generate_tokens(LM: LanguageModel, length=1000, start=()) -> List[Token]:
    """Generate a random text of `length` tokens, with an optional start, from `LM`."""
    tokens = list(start)
    while len(tokens) < length:
        history = cat(tokens[-LM.order:])
        tokens.append(random_sample(LM[history]))
    return tokens

Here are three auxiliary functions:
- `gen` is a convenience function to call `generate_tokens`, concatenate the resulting tokens, and print them.
- `random_sample` randomly chooses a single token from a Counter, with probability in proportion to its count.
- `cat` is a utility function to concatenate strings (tokens) into one big string.

In [5]:
def gen(LM: LanguageModel, length=1000, start=()) -> None:
    """Call generate_tokens and print the resulting tokens."""
    print(cat(generate_tokens(LM, length, start)))
    
def random_sample(counter: Counter) -> Token:
    """Randomly sample a token from the counter, proportional to each token's count."""
    return random.choices(list(counter), weights=list(counter.values()), k=1)[0]

cat = ''.join # Function to join strings together

Let's train a character-level language model of order 4 on the Shakespeare data. We'll call the model `LM`.  (Note that saying `tokens=data` means that the sequence of tokens is equal to the sequence of characters in `data`; in other words each character is a token.)

In [6]:
data = open("shakespeare_input.txt").read()

LM = train_LM(tokens=data, order=4)

Here are some examples of what's in the model, for various 4-character histories:

In [7]:
LM["chai"]

Counter({'n': 78, 'r': 35})

In [8]:
random_sample(LM["chai"])

'n'

In [9]:
LM["the "]

Counter({'p': 1360,
         's': 2058,
         'l': 1006,
         'o': 530,
         'g': 1037,
         'c': 1561,
         'a': 554,
         'C': 81,
         'r': 804,
         'h': 1029,
         'R': 45,
         'd': 1170,
         'w': 1759,
         'b': 1217,
         'm': 1392,
         'v': 388,
         't': 1109,
         'f': 1258,
         'i': 298,
         'n': 616,
         'V': 18,
         'e': 704,
         'u': 105,
         'L': 105,
         'y': 120,
         'A': 29,
         'H': 20,
         'k': 713,
         'M': 54,
         'T': 102,
         'j': 99,
         'q': 171,
         'K': 22,
         'D': 146,
         'P': 54,
         'S': 40,
         'G': 75,
         'I': 14,
         'B': 31,
         'W': 14,
         'E': 77,
         'F': 103,
         'O': 3,
         "'": 10,
         'z': 6,
         'J': 30,
         'N': 18,
         'Q': 7})

So `"chai"` is followed by either `'n'` or `'r'`, and almost any letter  can follow `"the "`.

## Generating Shakespeare

We cann generate a random text from the order 4 character model:

In [10]:
gen(LM)

First.

ROSALIND:
Warwick, but all house, stillow far into my ghost
prescience,
And will make the emperous in you gods!--One: that
Is ther,
that we stranglish, where assure, sir; a be king I will be accusers kind.

SIR HUGH EVANS:
Four grace
By so ye
As the admit trustice; but my wantony of thou did; and the shape Hectors:
Borest long.
Thou Marchs of him: his honour?
I this? hath is feed.

PANDARUS:
You anon her for you.

OCTAVIUS CAESAR:
You have saw to be a Jew, Orland would now welcomes no little prison! love the rest.

MARTIUS CAESAR:
Well, and the patience is adverthrowing me, I shall thee go thee whom I than to such and thy absolved
it, the ques
Weep a slighty sound,
Lychorse.

DESDEMONA:
This let us,
And so
On you, follow and epitation.

FALSTAFF:
Tell down.
These will luck'd out thee will teach of my apparis it, take of honest
Then for adors; the customary shot of that's place upstarved, ruin throne.

LAUNCE:
None lowly death,
Hath Parising upon all, bles, the live.

VIOLANUS:


Order 4 captures the structure of plays, mentions some characters, and generates mostly English words. But the words don't always go together to form grammatical sentences, and there is certainly no coherence or plot. 

## Generating Order 7 Shakespeare

What if we increase it to order 7? Or 10? We find that the output gets a bit better, roughly as good as the RNN models that Karpathy shows, and all from a much simpler *n*-gram model.

In [11]:
gen(train_LM(data, order=7))

First Citizen:
Which priest, Camillo,
I conjurations.

GOWER:
Here's Wart; thou at supper in't; and heart: but for that she shunn'd,
Nature of that are so many countryman, there's
another he folly could have spoke with me?

VINCENTIO:
You cannot cog, I can scarce any joy
Did ever saw a smith stand by the earth
I' the pedlar;
Money's a merited. These gloves are not palter it.

DUKE OF YORK:
I will go or tarrying you.

PRINCE HENRY:
Well, young son of Herne the shore the top,
And sigh a note for a king in the lean past your ambitious is the question: he swords:
Thanks, Rosencrantz and gentlemen: they are comes here.
Then, let grass grow dim. Fare you married Juliet?

LUCIANA:
Say, I would pardon. So it must be annoyance that thou shall fair praise
Have I something the lowest plant;
Whose in our army and unyoke.

BEDFORD:
'Twas very much content,
Clarence, be assure you will.

COMINIUS:
One word more: I am out--
That canst thou speak of Naples; 'twixt
The flight, never
Have writ to his ca

## Generating Order 10 Shakespeare

In [12]:
gen(train_LM(data, order=10))

First Citizen:
Why answer not?

ADRIANA:
By thee; and put a tongue,
That in civility and patience waiteth on true sorrow.
And see where comes another man, if I live.

TITUS ANDRONICUS:
Titus, prepare yourself in Egypt?

Soothsayer:
Beware the foul fiend! five
fiends have brought to be commander of the night gone by.

OCTAVIUS CAESAR:
O Antony, beg not your offence:
If you accept of grace: and from thy bed, fresh lily,
And when she was gone, but his judgment pattern of mine eyes;
Examine other be, some Florentine,
Deliver'd, both in one key,
As if our hands, our lives--

MACBETH:
I'll call them all encircle him about the streets?

First Gentleman:
That is intended drift
Than, by concealing it, heap on your way, which
is the way to Chester; and I would I wear them o' the city.

Citizens:
No, no, no; not so; I did not think,--
My wife is dead.
Your majesty is
returned with some few bands of chosen shot I had,
That man may question? You seem to have said, my noble and valiant;
For thou has

## Aside: Probabilities and Smoothing

Sometimes we'd rather see probabilities, not raw counts. Given a language model `LM`, the probability *P*(*t* | *h*) can be computed as follows:

In [13]:
def P(t, h, LM=LM): 
    "The probability that token t follows history h."""
    return LM[h][t] / sum(LM[h].values())

In [14]:
P('s', 'the ')

0.09286165508528112

In [15]:
P('n', 'chai')

0.6902654867256637

In [16]:
P('r', 'chai')

0.30973451327433627

In [17]:
P('s', 'chai')

0.0

In [18]:
P(' ', 'chai')

0.0

Shakespeare never wrote about "chaise longues," or "chai tea" so the probability of an `'s'` or `' '` following `'chai'` is zero, according to our language model. But do we really want to say it is absolutely impossible for the sequence of letters `'chais'` or `'chai '` to appear in a gebnerated text, just because we didn't happen to see it in our training data? More sophisticated language models use [**smoothing**](https://en.wikipedia.org/wiki/Kneser%E2%80%93Ney_smoothing) to assign non-zero (but small) probabilities to previously-unseen sequences. In this notebook we stick to the basic unsmoothed model.

## Aside: Starting Text

One thing you may have noticed: all the generated passages start the same. Why is that? Because the training data happens to start with the line "First Citizen:", and so when we call `generate_tokens`, we start with an empty history, and the only thing that follows the empty history in the training data is the letter "F", the only thing that follows "F" is "i", and so on, until we get to a point where there are multiple choices. We could get more variety in the start of the generated text by breaking the training text up into multiple sections, so that each section would contribute a different possible starting point. But that would require some knowledge of the structure of the training text; right now the only assumption is that it is a sequence of tokens/characters.

We can give a starting text to `generate_tokens` and it will continue from there. But since the models only look at a few characters of history (just 4 for `LM`), this won't make much difference. For example, the following won't make the model generate a story about Romeo:

In [19]:
gen(LM, start='ROMEO')

ROMEO:
Ay, strong;
A little. Prince of such matten a virtue's states:
Now country's quent again,
A most unacceptre her no leave-thou true shot, and, to
things of mine hath done alehousand pride
For be, whithee thus with of damm'st fly and came which he rank,
Put not wages woman! Harform
So long agon's side,
Singer's me her my slinged! He wit:' quoth Dorse! art than manner
straine.

AGAMEMNON:
I would says.

BEATRICE:
Between my secreason ragined our uncrowned justing in thither naturer?

GLOUCESTER:
The king.

DOGBERRY:
Do now nothink you
The tired.

HORATIAN:
It stray
To see, peers, eo me, while; this days this man carbona-mi's, and for
in this late ther's loath,
Diminutes, shepher, sir.

BARDOLPH:
So.

PETRUCHIO:
'Tis Angels upon me?
Hath no fare on,
what's the
moods.

KING RICHARLES:
Thou odourse first Servantas, given in the life' thee, he same world I have spirit,
Or lord Phoebus' Corne.

KENT:
Go you shafts that satisfied; leaven blession what charge of a this: for.

ACHIMO:
'Zou

# Linux Kernel C++ Code

Goldberg's point is that the simple character-level n-gram model performs about as well as the  more complex RNN model on Shakespearean text. 

But Karpathy also trained an RNN on 6 megabytes of Linux-kernel C++ code. Let's see what we can do with that training data.

In [20]:
! [ -f linux_input.txt ] || curl -O https://norvig.com/ngrams/linux_input.txt
! wc linux_input.txt

  241465  759639 6206997 linux_input.txt


In [21]:
linux = open("linux_input.txt").read()

## Generating Order 10 C++

We'll start with an order-10 character model, and compare that to an order-20 model. We'll generate a longer text, because sometimes 1000 characters ends up being just one long comment.

In [22]:
gen(train_LM(linux, order=10), length=3000)

/*
 * linux/kernel.h>
#include <linux/security.h>
#include <linux/syscalls.h>
#include <linux/ptrace.h>
#include <linux/ftrace_event_file *file)
{
	struct tracer *t)
{
	while (!list_empty(&op->list, &ap->list) && kprobe_disarmed(p))
		return;

	/*
	 * Actually logging the members for stats sorting */
	int			work_color = next_color;
				atomic_inc(&bt->dropped, 0);
	INIT_LIST_HEAD(&rp->free_instance *pinst, int cpu)
{
	struct list_head *timers = tsk->cpu_timers_exit.
 */
void __weak arch_show_interrupt(), "Trying to figure out
	 * to the pending nohz_balance_enter_idle(void) { return -EAGAIN;
}

/**
 * irq_domain *domain;

	domain = irq_data_to_desc(data);

	if (unlikely(iter->cache_reader_page->list, pages);
	swsusp_show_speed(start, end = ULLONG_MAX;
		return;

preempt:
	resched_curr(rq);
#endif
		siginitset(&flush, sigmask(SIGSTOP));
	__set_current_state(TASK_INTERRUPTS)
		return -EINVAL;
		error = prepare_image(&orig_bm, &copy_bm);

	while (ptr < buf + buf_len) {
		ab = audit_log_lo

## Order 20 C++

In [23]:
gen(train_LM(linux, order=20), length=3000)

/*
 * linux/kernel/irq/pm.c
 *
 * Copyright (C) 2004 Pavel Machek <pavel@ucw.cz>
 * Copyright (C) 2007-2008 Steven Rostedt <srostedt@redhat.com>
 * Copyright (C) 2014 Seth Jennings <sjenning@redhat.com>
 * Copyright (C) 2005-2006 Thomas Gleixner
 *
 *  No idle tick implementation for gcov data files. Release resources allocated
 * by open().
 */
static int gcov_seq_release(struct inode *inode, struct file *file)
{
	put_event(file->private_data);
	}

	return 0;

free:
	klp_free_funcs_limited(obj, NULL);
		kobject_put(obj->kobj);
	}
}

static void print_other_cpu_stall(rsp, gpnum);
	}
}

/**
 * rcu_cpu_stall_reset(void)
{
	struct tick_sched *ts)
{
	ktime_t now = ktime_get();

	if (ts->idle_active && nr_iowait_cpu(cpu) > 0) {
			ktime_t delta = ktime_sub(now, alarm->node.expires,
							incr*overrun);

		if (alarm->node.expires,
							incr*overrun);

		if (alarm->node.expires,
				HRTIMER_MODE_ABS);
		if (!hrtimer_active(&t.timer))
		t.task = NULL;

	if (likely(!audit_ever_enabled))
		re

## Analysis of Generated Linux Text

As Goldberg says, "Order 10 is pretty much junk." But order 20 is much better. Most of the comments have a start and an end; most of the open parentheses are balanced with a close parentheses; but the braces are not as well balanced. That shouldn't be surprising. If the span of an open/close parenthesis pair is less than 20 characters then it can be represented within the model, but if the span of an open/close brace is more than 20 characters, then it cannot be represented by the model. Goldberg notes that Karpathy's RRN seems to have learned to devote some of its long short-term memory (LSTM) to representing nesting level, as well as things like whether we are currently within a string or a comment. It is indeed impressive, as Karpathy says, that the model learned to do this on its own, without any input from the human engineer.

## Token Models versus Character Models

Karpathy and Goldberg both used character models, because the exact formatting of characters (especially indentation and line breaks) is important in the format of plays and C++ programs. But if you are interested in generating paragraphs of text that don't have any specific format, it is  common to use a **word** model, which represents the probability of the next word given the previous words, or a **token** model in which tokens can be words, punctuation, or parts of words. For example, the text `"Spiderman!"` might be broken up into the three tokens `"Spider"`, `"man"`, and `"!"`. 

One simple way of tokenizing a text is to break it up into alternating word and non-word characters; the function `tokenize` does that by default:

In [24]:
import re

def tokenize(text: str, regex=r'\w+|\W+') -> List[Token]: 
    """Break text up into tokens using regex; 
    by default break into alternating word-character and non-word-character tokens."""
    return re.findall(regex, text)

In [25]:
assert tokenize('Soft! who comes here?') == [
    'Soft', '! ', 'who', ' ', 'comes', ' ', 'here', '?']

We can train a token model on the Shakespeare data. A model of order 6 keeps a history of up to three word and three non-word tokens. The keys of the language model dictionary consist of strings formed by concatenating the 6 tokens together.

In [26]:
TLM = train_LM(tokenize(data), order=6)

In [27]:
TLM['wherefore art thou ']

Counter({'Romeo': 1})

In [28]:
TLM['not in our ']

Counter({'stars': 1, 'Grecian': 1})

In [29]:
TLM['end of my ']

Counter({'life': 1, 'business': 1, 'dinner': 1, 'time': 1})

In [30]:
TLM[' end of my']

Counter({' ': 2})

We see below that the quality of the token models is similar to character models, and improves from 6 tokens to 8:

In [31]:
gen(TLM)

First Citizen:
Before we proceed any further, hear me speak.

CASSIO:
Madam, not now: I am glad to hear you tell my worth
Than you much willing to be counted wise
In spending your wit in the praise.

AJAX:
I do hate him
As rootedly as I. Burn but his books.
He has brave utensils,--for so he calls me: now I feed myself
With most delicious poison. Think on me,
That cropp'd the golden prime of this sweet prince,
And if your grace mark every circumstance,
You have great reason to do Richard right;
Especially for those occasions
At Eltham Place I told your majesty as much before:
This proveth Edward's love and Warwick's, and must have my will;
If one good deed in a naughty world.

NERISSA:
When the moon shone, we did not see your grace.

DUKE:
I am sorry, madam, I have hurt your kinsman:
But, had it been a carbuncle
Of Phoebus' wheel, and might so safely, had it
Been all the worth of man's flesh taken from a man
Is not so estimable, profitable neither,
As flesh of muttons, beefs, or goats. 

In [32]:
gen(train_LM(tokenize(data), 8))

First Citizen:
Before we proceed any further, hear me speak.

All:
Peace, ho! Hear Antony. Most noble Antony!

ANTONY:
Why, friends, you go to do you know not what:
Wherein hath Caesar thus deserved your loves?
Alas, you know not: I must tell you then:
You have forgot the will I told you of.

All:
Most true. The will! Let's stay and hear the will.

ANTONY:
Here is the will, and under Caesar's seal.
To every Roman citizen he gives,
To every several man, seventy-five drachmas.

Second Citizen:
Most noble Caesar! We'll revenge his death.

Third Citizen:
O royal Caesar!

ANTONY:
Hear me with patience.

IMOGEN:
Talk thy tongue weary; speak
I have heard I am a strumpet; and mine ear
Therein false struck, can take no greater wound,
Nor tent to bottom that. But speak.

PISANIO:
Then, madam,
I thought you would not back again.

IMOGEN:
Most like;
Bringing me here to kill me.

PISANIO:
Not so, neither:
But if I were as tedious as go o'er:
Strange things I have in head, that will to hand;
Which m

## C++ Token Model

Similar remarks hold for token models trained on C++ data:

In [33]:
gen(train_LM(tokenize(linux), 8), length=3000)

/*
 * linux/kernel/irq/autoprobe.c
 *
 * Copyright (C) 1992, 1998-2006 Linus Torvalds, Ingo Molnar
 *
 * This file contains spurious interrupt handling.
 */

#include <linux/jiffies.h>
#include <linux/math64.h>
#include <linux/timex.h>
#include <linux/export.h>
#include <linux/spinlock.h>
#include <linux/smp.h>
#include <linux/bug.h>
#include <linux/cpumask.h>
#include <linux/cpu.h>
#include <linux/err.h>
#include <linux/hrtimer.h>
#include <linux/tick.h>
#include <linux/cpu.h>
#include <linux/err.h>
#include <linux/init.h>
#include <linux/errno.h>
#include <linux/module.h>
#include <linux/proc_fs.h>
#include <linux/seq_file.h>
#include <linux/time.h>
#include <linux/hrtimer.h>
#include <linux/interrupt.h>
#include <linux/device.h>
#include <linux/clocksource.h>
#include <linux/ring_buffer.h>
#include <generated/utsrelease.h>
#include <linux/stacktrace.h>
#include <linux/debug_locks.h>
#include <linux/perf_event.h>

/*
 * The run state of the lockup detectors.
	 */
	if (!write) {
		*wa